## Submission Notebook Template 

<b> Install your packages below: </b>

In [ ]:
!pip install git+https://github.com/Total-RD/pymgrid/
## Other packages 

In the section below, you must run your methodology for solving the problem from start to finish :

In [ ]:
import pickle

"""
The buildings mentionned below are specific to the hackathon and are not available in this repo.
You can replace them with any MicroGrid object generated from pymgrid
"""

with open('building_1.pkl', 'rb') as f:
    building_1 = pickle.load(f)
    building_1.train_test_split()

with open('building_2.pkl', 'rb') as f:
    building_2 = pickle.load(f)
    building_2.train_test_split()

    
with open('building_3.pkl', 'rb') as f:
    building_3 = pickle.load(f)
    building_3.train_test_split()

    
buildings = [building_1, building_2, building_3]

<b> 1) Import all used libraries and scripts here </b>

In [ ]:
import time # Necessary to evaluate frugality
from pymgrid.Environments.pymgrid_cspla import MicroGridEnv # Imposed Environment
import numpy as np

## Import your favourite Deep Learning library for RL and other packages here
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

<b> 2) Agent & Environment Setup before your training </b>

In [ ]:
"""
Below is an environment initialization without a Deep RL library, the code can vary depending on which library you 
use
"""

#Class that we use to optimize our model of agents

class OPT:
    def __init__(self, Nboucle, nb_steps, env1, env2, env3):
        
        """It initializes the models for the agents and the agents itself. 
    This object can be opitimized thanks to an object method with the hyperparameters
    Nboucle and nb_steps that are self explanatory.
    """
        #Hyperparameters
        self.Nboucle = Nboucle
        self.nb_steps =  nb_steps
        
        #Model for building 1
        self.env1 = env1
        states1 = env1.observation_space.shape
        actions1 = env1.action_space.n
        self.model1 = self.build_model12(states1, actions1)
        self.dqn1 = self.build_agent(self.model1, actions1)
        
        #Model for building 2        
        self.env2 = env2
        self.model2 = self.build_model12(states1, actions1)
        self.dqn2 = self.build_agent(self.model2, actions1)
        
        #Model for building 3
        self.env3 = env3        
        states3 = env3.observation_space.shape
        actions3 = env3.action_space.n
        self.model3 = self.build_model3(states3, actions3)
        self.dqn3 = self.build_agent(self.model3, actions3)
        
        
    def optBuilding(self,lr_init, decay_steps, Nboucle, index, model, dqn, env):
        """It optimizes the agent 'dqn' of the building 'index' with the hyperparameters in argument"""
        #Initial learning rate
        lr = tf.keras.optimizers.schedules.ExponentialDecay(lr_init, decay_steps, 0.65, staircase=False, name=None)
        dqn.compile(Adam(learning_rate =lr), metrics=['mae'])
        print(f"Building {index}: ",'\n')
        dqn.fit(env, nb_steps=self.nb_steps*Nboucle, visualize=False, verbose=1)
        print('\n')

            
            
    def opt(self, name=''):
        """It allows us to optimize every agents and save the results in a file .h5f to reuse the results."""    
        #optimization Building 1
        self.optBuilding(0.1, 3000, self.Nboucle, 1, self.model1 ,self.dqn1, self.env1)
        print()
        self.dqn1.save_weights(f'{name}_weights1.h5f', overwrite=True)
        
        #optimization Building 2
        self.model2.load_weights(f'{name}_weights1.h5f') #transfer learning
        self.optBuilding(0.1, 3000, self.Nboucle, 2, self.model2 ,self.dqn2, self.env2)
        print()
        
        #optimization Building 3
        self.optBuilding(0.1 , 5000, 2*self.Nboucle, 3, self.model3 ,self.dqn3, self.env3)
        print()
    
        self.save(name)
        
    def test(self):
        """This function is testing the three models and return each profitability"""    
    
        lr=0.005
        self.dqn1.compile(Adam(lr=lr), metrics=['mae'])
        self.dqn2.compile(Adam(lr=lr), metrics=['mae'])
        self.dqn3.compile(Adam(lr=lr), metrics=['mae'])

        
        scores = self.dqn1.test(self.env1, visualize=False)
        score1 = -np.mean(scores.history['episode_reward'])
        
        scores = self.dqn2.test(self.env2, visualize=False)
        score2 = -np.mean(scores.history['episode_reward'])

        scores = self.dqn3.test(self.env3, visualize=False)
        score3 = -np.mean(scores.history['episode_reward'])
        
        return score1, score2, score3
                
        
    def save(self, name=''):
        """It allows us to save our results with a parameter for the name called 'name '. """
        self.dqn1.save_weights(f'{name}_weights1.h5f', overwrite=True)
        self.dqn2.save_weights(f'{name}_weights2.h5f', overwrite=True)
        self.dqn3.save_weights(f'{name}_weights3.h5f', overwrite=True)

    
    def load(self, name):
        """We can load the three models with the information 'name' used to save them."""
        #Loading of models
        self.model1.load_weights(f'{name}_weights1.h5f')
        self.model2.load_weights(f'{name}_weights2.h5f')
        self.model3.load_weights(f'{name}_weights3.h5f')

        #Inputs
        actions1 = self.env1.action_space.n
        actions3 = self.env3.action_space.n
        
        #Modification of the agents
        self.dqn1 = self.build_agent(self.model1, actions1)
        self.dqn2 = self.build_agent(self.model2, actions1)
        self.dqn3 = self.build_agent(self.model3, actions3)
           

    def build_model12(self, states, actions):
        """This function returns the model use for the agent of the building 1 and 2."""
        model = Sequential()
        model.add(Flatten(input_shape=(1,states[0])))
        model.add(Dense(60, activation='relu' , input_shape=(1,states[0])))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(actions, activation='linear'))
        return model
    
    def build_model3(self, states, actions):
        """This function returns the model use for the agent of the building 3."""
        model = Sequential()
        model.add(Flatten(input_shape=(1,states[0])))
        model.add(Dense(90, activation='relu' , input_shape=(1,states[0])))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(actions, activation='linear'))
        return model
    
    def build_agent(self,model, actions):
        """It builds an double DQN agent that move with BoltzmannQPolicy"""
        policy = BoltzmannQPolicy()
        memory = SequentialMemory(limit=8760, window_length=1)
        dqn = DQNAgent(model=model, enable_double_dqn=True, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=500, target_model_update=1e-6)

        return dqn 

In [ ]:
#Environments creations
building_environments = [MicroGridEnv(env_config={'microgrid':buildings[i]}) for i in range(3)]
building_environments[0].reset(testing=False)
building_environments[1].reset(testing=False)
building_environments[2].reset(testing=False)

for i in range(3):
    building_environments[i].reset(testing=False)
    building_environments[i].seed(42)
      

#Environments notations
env1 = building_environments[0]
env2 = building_environments[1]
env3 = building_environments[2]




<b> 3) Training of the agent </b>

In [ ]:
train_start = time.process_time()

#Hyperparameters
Nboucle = 7
nb_steps = 10000

#Creation of the models
ModelBuildings = OPT(Nboucle, nb_steps, env1, env2, env3)

#Optimization and save of the parameters found
ModelBuildings.opt('Train_7_')

train_end = time.process_time()
train_frugality = train_end - train_start

<b> 4) Test of the agent </b>

In [ ]:


test_start = time.process_time()

for i in range(3):
    building_environments[i].reset(testing=False)
    building_environments[i].seed(42)

env1 = building_environments[0]
env2 = building_environments[1]
env3 = building_environments[2]

ModelBuildings = OPT(Nboucle, nb_steps, env1, env2, env3)
ModelBuildings.load('Train_7_')

total_cost = ModelBuildings.test()

test_end = time.process_time()

test_frugality = test_end - test_start

<b> 5) Store & Export Results in JSON format </b>

In [ ]:
final_results = {
    "building_1_performance" : total_cost[0],
    "building_2_performance" : total_cost[1],
    "building_3_performance" : total_cost[2],
    "frugality" : train_frugality + test_frugality,
}
print(final_results)